## 탐색 알고리즘 (실패)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from random import randint

In [ ]:
def mse(img1, img2):
    h, w = img1.shape
    diff = cv2.subtract(img1, img2)
    err = np.sum(diff**2)
    mse = err / (float(h * w))
    return mse, diff

In [ ]:
# Load the input videos
test = cv2.VideoCapture('test1.mp4')
origin = cv2.VideoCapture('test2.mp4')

In [ ]:
# Get video properties (assuming both videos have the same properties)
fps = int(test.get(cv2.CAP_PROP_FPS))
frame_width = int(test.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(test.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
# Divide the frame into 9 regions
regions = [
    ((0, frame_width // 3), (0, frame_height // 3)),
    ((frame_width // 3, 2 * frame_width // 3), (0, frame_height // 3)),
    ((2 * frame_width // 3, frame_width), (0, frame_height // 3)),
    ((0, frame_width // 3), (frame_height // 3, 2 * frame_height // 3)),
    ((frame_width // 3, 2 * frame_width // 3), (frame_height // 3, 2 * frame_height // 3)),
    ((2 * frame_width // 3, frame_width), (frame_height // 3, 2 * frame_height // 3)),
    ((0, frame_width // 3), (2 * frame_height // 3, frame_height)),
    ((frame_width // 3, 2 * frame_width // 3), (2 * frame_height // 3, frame_height)),
    ((2 * frame_width // 3, frame_width), (2 * frame_height // 3, frame_height))
]

In [ ]:
frameCounter = 0  # Initialize frame counter

# Define the number of regions for comparison
num_regions = 9

# Define the size of each region (percentage of frame size)
region_size = 0.2

# Get the size of the frame
frame_width = int(test.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(test.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Resize region size based on the frame size
region_height = int(frame_height * region_size)
region_width = int(frame_width * region_size)

In [ ]:
while True:
    # Read frames from both videos
    ret1, frame1 = test.read()
    ret2, frame2 = origin.read()

    # If any of the videos has reached the end, break the loop
    if not ret1 or not ret2:
        break
    
    frameCounter += 1  # Increment frame counter

    # Compare frames only every 30 frames
    if frameCounter % 30 == 0:
        # Convert frames to grayscale
        grayFrame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        grayFrame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Get the center region of video1 for comparison
        center_x1 = grayFrame1.shape[1] // 3
        center_x2 = center_x1 * 2
        center_y1 = grayFrame1.shape[0] // 3
        center_y2 = center_y1 * 2
        center_region_frame1 = grayFrame1[center_y1:center_y2, center_x1:center_x2]
        
        # Compute the size of each region in video2
        region_height = int(center_region_frame1.shape[0] * region_size)
        region_width = int(center_region_frame1.shape[1] * region_size)

        # Extract multiple regions from video2 for comparison
        regions_frame2 = []
        for i in range(num_regions):
            x1 = int((i + 0.5) * (grayFrame2.shape[1] - region_width) / num_regions)
            x2 = x1 + region_width
            y1 = int((i + 0.5) * (grayFrame2.shape[0] - region_height) / num_regions)
            y2 = y1 + region_height
            region_frame2 = grayFrame2[y1:y2, x1:x2]
            regions_frame2.append(region_frame2)

        # Compute MSE for each region in video1 and regions in video2
        mse_values = []
        for region_frame2 in regions_frame2:
            center_region_frame1 = cv2.resize(center_region_frame1, (region_frame2.shape[1], region_frame2.shape[0]))
            error, diff = mse(center_region_frame1, region_frame2)
            mse_values.append(error)

        # Check if the center region of video1 belongs to video2
        threshold = 75  # Adjust this threshold based on your data
        is_center_included = any(mse_value > threshold for mse_value in mse_values)

        # Visualization (You can modify this part as needed)
        plt.figure(figsize=(15, 10))
        plt.subplot(2, 2, 1)
        plt.imshow(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))
        plt.title(f"Test Video")
        plt.axis('off')

        plt.subplot(2, 2, 2)
        plt.imshow(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB))
        plt.title(f"Origin Video - Center Included: {is_center_included}")
        plt.axis('off')
        
        plt.subplot(2, 2, 3)
        plt.imshow(cv2.cvtColor(center_region_frame1, cv2.COLOR_BGR2RGB))
        plt.title(f"Test Video - Cropped")
        plt.axis('off')
        
        plt.subplot(2, 2, 4)
        plt.imshow(cv2.cvtColor(regions_frame2[randint(0, num_regions - 1)], cv2.COLOR_BGR2RGB))
        plt.title(f"Origin Video - Cropped")
        plt.axis('off')

        plt.show()
        clear_output(wㅠait=True)

In [ ]:
# Release video capture objects
test.release()
origin.release()

## SIFT (성공)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [ ]:
def find_similar_points(video1Path, video2Path):
    # Load the input videos
    video1 = cv2.VideoCapture(video1Path)
    video2 = cv2.VideoCapture(video2Path)

    # SIFT 특징점 검출
    sift = cv2.SIFT_create()
    
    frameCounter = 0

    while True:
        # Read frames from both videos
        ret1, frame1 = video1.read()
        ret2, frame2 = video2.read()

        # If any of the videos has reached the end, break the loop
        if not ret1 or not ret2:
            break
            
        frameCounter += 1  # Increment frame counter

        # Compare frames only every 30 frames
        if frameCounter % 180 == 0:

            # Convert frames to grayscale
            grayFrame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            grayFrame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

            # SIFT 특징점 검출 및 매칭
            kp1, des1 = sift.detectAndCompute(grayFrame1, None)
            kp2, des2 = sift.detectAndCompute(grayFrame2, None)

            # 특징점 매칭
            bf = cv2.BFMatcher()
            matches = bf.knnMatch(des1, des2, k=2)

            # 거리가 가까운 매칭 결과 선택
            goodMatches = []
            for m, n in matches:
                if m.distance < 0.75 * n.distance:
                    goodMatches.append(m)

            # 유사점 그리기
            resultImg = cv2.drawMatches(grayFrame1, kp1, grayFrame2, kp2, goodMatches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
            
            title = f"Number of similarities: {len(goodMatches)}" if len(goodMatches) > 300 else f"Number of similarities: {len(goodMatches)}\nGet off the road"

            # Show the result
            plt.figure(figsize=(15, 10))
            plt.imshow(resultImg)
            plt.title(title)
            plt.axis('off')
            plt.show()
            clear_output(wait=True)

    # Release video capture objects
    video1.release()
    video2.release()

In [ ]:
if __name__ == "__main__":
    video1_path = "test1.mp4"
    video2_path = "test2.mp4"
    find_similar_points(video1_path, video2_path)